# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [31]:
import numpy as np 
import pandas as pd
from scipy.stats import trim_mean, mode, skew, gaussian_kde, pearsonr, spearmanr, beta
from statsmodels.stats.weightstats import ztest as ztest
from scipy.stats import ttest_ind, norm, t
from scipy.stats import f_oneway
from scipy import stats



# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents

In [18]:
salaries = pd.read_csv('../Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examine the `salaries` dataset using the `head` function below.

In [19]:
salaries.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [20]:
salaries.isna().sum()

Name                     0
Job Titles               0
Department               0
Full or Part-Time        0
Salary or Hourly         0
Typical Hours        25161
Annual Salary         8022
Hourly Rate          25161
dtype: int64

Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [21]:
salaries['Salary or Hourly'].value_counts()



Salary or Hourly
Salary    25161
Hourly     8022
Name: count, dtype: int64

What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [23]:
department_counts = salaries.groupby('Department').size().reset_index(name='Employee_Count')
department_counts

,Department,Employee_Count
0,ADMIN HEARNG,39
1,ANIMAL CONTRL,81
2,AVIATION,1629
3,BOARD OF ELECTION,107
4,BOARD OF ETHICS,8
5,BUDGET & MGMT,46
6,BUILDINGS,269
7,BUSINESS AFFAIRS,171
8,CITY CLERK,84
9,CITY COUNCIL,411


# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [25]:
# z-test

data_hourly = salaries[salaries["Salary or Hourly"] == "Hourly"]["Hourly Rate"]
alpha = 0.05

_, p_value = ztest(data_hourly, value = 30, alternative='two-sided')
p_value


1.8228873859286195e-94

In [26]:
h0 = "H0: Hourly wage is not significally different than $30/hr. We keep H0. Not enough info"
h1 = "H1: Hourly wage is significally different than $30/hr. We reject H0 and keep H1."

if p_value < alpha: # It is very unlikely (p value is very low) that it happens, given  H0 is true 
    print(h1)
else:
    print(h0)

H1: Hourly wage is significally different than $30/hr. We reject H0 and keep H1.


We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [27]:
# z-test

data_police = salaries[salaries["Department"] == "POLICE"]["Annual Salary"].dropna()
alpha = 0.05

_, p_value = ztest(data_police, value = 86000, alternative='larger')
p_value

0.0010280845287547855

In [28]:
h0 = "H0: Salaries are not significally different than $86000/year. We keep H0. Not enough info."
h1 = "H1: Salaries are significally different than $86000/year. We reject H0 and keep H1."

if p_value < alpha: # It is very unlikely (p value is very low) that it happens, given  H0 is true 
    print(h1)
else:
    print(h0)

H1: Salaries are significally different than $86000/year. We reject H0 and keep H1.


Using the `crosstab` function, find the department that has the most hourly workers. 

In [12]:
cross_table = pd.crosstab(salaries['Department'], salaries['Salary or Hourly'])

# Select only the 'Hourly' column and find the department with the most hourly workers
department_most_hourly = cross_table['Hourly'].idxmax()

print(f"The department with the most hourly workers is: {department_most_hourly}")


The department with the most hourly workers is: STREETS & SAN


The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [32]:
data_streets = salaries[salaries["Department"] == "STREETS & SAN"]["Hourly Rate"].dropna()

t_stat, p_value = stats.ttest_1samp(data_streets,35,alternative='less' )
alpha = 0.05  

h0 = "H0: Hourly wage is not significally different than $35/hr. We keep H0. Not enough info"
h1 = "H1: Hourly wage is significally different than $35/hr. We reject H0 and keep H1."

if p_value < alpha: # It is very unlikely (p value is very low) that it happens, given  H0 is true 
    print(h1)
else:
    print(h0)


H1: Hourly wage is significally different than $35/hr. We reject H0 and keep H1.


# Challenge 3 - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [33]:
hourly_wage_data = salaries['Hourly Rate'].dropna()

# Calculate the mean and standard error
mean_hourly_wage = hourly_wage_data.mean()
standard_error = sem(hourly_wage_data)

# Set the confidence level and degrees of freedom
confidence_level = 0.95
degrees_of_freedom = len(hourly_wage_data) - 1

# Compute the confidence interval
confidence_interval = t.interval(confidence_level, degrees_of_freedom, mean_hourly_wage, standard_error)

print(f"95% Confidence Interval for Mean Hourly Wage: ({round(confidence_interval[0], 2)}, {round(confidence_interval[1], 2)})")

95% Confidence Interval for Mean Hourly Wage: (32.52, 33.05)


Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [15]:
police_salary_data = salaries.loc[(salaries['Department'] == 'POLICE') & (salaries['Salary or Hourly'] == 'Salary'), 'Annual Salary'].dropna()

# Calculate the mean and standard error
mean_police_salary = police_salary_data.mean()

standard_error_police = sem(police_salary_data)

# Set the confidence level and degrees of freedom
confidence_level = 0.95
degrees_of_freedom_police = len(police_salary_data) - 1

# Compute the confidence interval
confidence_interval_police = t.interval(confidence_level, degrees_of_freedom_police, mean_police_salary, standard_error_police)

print(f"95% Confidence Interval for Mean Annual Salary in the Police Department: ({round(confidence_interval_police[0], 2)}, {round(confidence_interval_police[1], 2)})")

95% Confidence Interval for Mean Annual Salary in the Police Department: (86177.06, 86795.77)


# Challenge 4 - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [35]:
hourly_workers = salaries[salaries['Salary or Hourly'] == 'Hourly']

# Count the total number of workers and the number of hourly workers
total_workers = len(salaries)
hourly_workers_count = len(hourly_workers)

# Set the null hypothesis proportion
null_hypothesis_proportion = 0.25  # 25%

# Perform proportions z-test
z_stat, p_value = proportions_ztest(hourly_workers_count, total_workers, null_hypothesis_proportion)

alpha = 0.05  # 95% confidence level
print(f'P-value: {p_value}')

h0 = "H0: Number of hourly workers in the City of Chicago is not significantly different from 25%. We keep H0. Not enough info."
h1 = "H1: Number of hourly workers in the City of Chicago is significantly different from 25%. We reject H0 and keep H1."

if p_value < alpha: # It is very unlikely (p value is very low) that it happens, given  H0 is true 
    print(h1)
else:
    print(h0)


P-value: 0.0004481127249057967
H1: Number of hourly workers in the City of Chicago is significantly different from 25%. We reject H0 and keep H1.
